In [3]:
import pandas as pd
import json

In [4]:
import pandas as pd
import re

sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
groups = []
with open("CS_Word_20240405_22.25.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if matched == True and last_line[-1] != "。":
            sentence["content"] += line
            last_line = line
            continue

        if sentence_content_pattern.match(line.strip()):
            matched = True
            sentence = {"content": sentence_content_pattern.sub(r"\1 ", line)}
        else:
            if matched:
                sentence["hiragana"] = ""
                sentence["meaning"] = line
                if sentence["meaning"][1] == " ":
                    sentence["meaning"] = sentence["meaning"][2:]

                if sentence["meaning"][0] == "囉":
                    sentence["meaning"] = sentence["meaning"][1:]

                if sentence["content"][0] == "①" and groups:
                    example_sentences.append(groups)
                    groups = []

                groups.append(sentence)
            matched = False
        last_line = line

if groups:
    example_sentences.append(groups)

print(example_sentences[1])
with open("example_sentences.json", "w", encoding="utf-8") as fp:
    fp.write(json.dumps(example_sentences, ensure_ascii=False, indent=4))
    # pd.DataFrame(example_sentences, dtype=str).fillna("").to_json(fp, orient="records", force_ascii=False, indent=4)

[{'content': '① わたしはその人の顔を一目見るや、 30年前に別れた恋人だと気がついた。', 'hiragana': '', 'meaning': '我刚看了那个人一眼,就认出了他是我三十年前分手的恋人。'}, {'content': '② 子供たちは唐揚げが大好物で、 食卓に出すや、あっという間になくなってしまう。', 'hiragana': '', 'meaning': '孩子们非常喜欢炸鸡块,刚摆到桌子上,一眨眼的工夫就没了。'}, {'content': '③ 選挙戦が始まるや否や、 あちこちからにぎやかな声が聞こえてきた。', 'hiragana': '', 'meaning': '选举战刚一开始,四处就开始喧闹起来了。'}, {'content': '④ この病気の新しい治療法が発表されるや否や、 全国の病院から問い合わせが殺到した。', 'hiragana': '', 'meaning': '这种疾病的治疗法刚一公布,来自全国医院的问讯就蜂拥而至。'}]


'B'

In [16]:
import pandas as pd

df = pd.read_json("grammar.json", dtype=str)
df.head()
for index in df.index:
    parts = [part for part in df.loc[index, "meaning"].split("\n") if part.strip()]
    if len(parts) != 2:
        df.loc[index, "japanese_meaning"] = "\n".join(parts[0::2])
        df.loc[index, "chinese_meaning"] = "\n".join([f"{chr(ord('A') + idx)} {meaning}" for idx, meaning in enumerate(parts[1::2])])
    else:
        df.loc[index, "japanese_meaning"] = parts[0]
        df.loc[index, "chinese_meaning"] = parts[1]
    print("=" * 32)
    print(df.loc[index, "japanese_meaning"])
    print(df.loc[index, "chinese_meaning"])
    print("=" * 32)

~とき 硬い言い方
（…………时候。书面语。）
⇒~するとき 硬い言い方
当・・・・・・ 的時候。书面語。
⇒~したら、直後に意外なことが起こる。
做了…………之后,紧接着发生意外的事情。
⇒ 〜の後、すぐに続いて次の出来事や大きな変化が起こる。
……之后,紧接着马上发生下一个事件或重大变化。
⇒~が終わると同時に、次のことが起こる。
………结束的同时,发生下一件事情。
⇒ちょうど~しているところだ。
正在做......的时候。
A⇒時間の制限があって、 〜でなくなった後では実現が難しいから、その前にしてしまう。
B ⇒ ~している間に変化が現れる。
A 因为有时间限制,如果变得不......之后那就很难实现,所以要赶在这种变化发生之前完成。
B 正在做......期间发生了变化。
⇒~という一方方向に変化が進んでいく。
变化朝着......单一方向不断加剧。
⇒ 〜という変化が起こる少し前だ・もうすぐ〜する。硬い言い方
在……这个变化发生的前一刻、马上就要...……。书面语。
⇒~という変化が進行中だ。 硬い言い方
…………变化正在进行过程中。书面语。
⇒ 〜ながら、 あることをする。硬い言い方
一边……,一边做某事。书面语。
⇒~を経験した後や、 〜という状態になった後で、 今までになかったことが起こる。
经历……以后,或者变成…………状态以后,发生了以前未出现过的事情。
⇒準備としてまず〜してから、 その後で次に続く行動をする。
作准先做......, 其后再做接下来的行為。
⇒ 〜が実現した後、 すぐに続けてある行動をする。硬い言い方
…………实现以后,马上采取后续行为。书面语。
⇒~してから今まで、ずっと同じ状態が続いている。
做…………之后直到现在,一直持续着相同的状态。
⇒~した後でなければ、あることが実現しない。
如果不是做完……以后的话,某事就不会实现。
⇒ 〜が代表例で、 そのほかにもいろいろある。硬い言い方
…………是有代表性的例子,除此以外还有很多其他的。书面语。
⇒ 〜という例一つを取ってもそうなのだから、 全体的にももちろんそうだ。
即使只抽取………..这一个例子都是这样的,所以整体来看也当然如此。
⇒ 〜の範囲全体にその状態が広がっている。
其状态扩大到……的全部。
A 〜の期間ずっと同じ状態だ。
B⇒ ~を手段にして、あることをする。

In [ ]:
import text_to_audio

In [ ]:
text_to_audio.generate_audio([
    (None, "このマラソン大会には、性別、年齢を問わず"),
], None)

In [5]:
grammar_content_pattern = re.compile(r"^\d? ?[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")

grammar_content_pattern.findall("~ 際 (に)")

['~ 際 (に)']

In [42]:
content = "ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"
content = re.sub("\s+", " ", content)
print(content)

question_pattern = re.compile(r"[（(](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[)）]")
print(question_pattern.findall(content))

ここに車を止められるのは、許可をもらっている人（a だけ b に限り）です。
['（a だけ b に限り）']


In [21]:
re.findall(r"(?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+", "a だけ b に限り")

['a だけ b に限り']

In [10]:
question_pattern = re.compile(r"[(（](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[）)]*\?")
print(question_pattern.findall("こちらの会議室をご利用になる際は、 受付で必要事項をご記入ください。"))
print(question_pattern.findall("ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"))

[]
[]


In [ ]:
import pandas as pd
import re

grammar_content_pattern = re.compile(r"^\d [~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")
sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
with open("CS_Word_20240403_23.41.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if grammar_content_pattern.match(line):
            print(line)